In [12]:
import pandas as pd
import numpy as np
import yfinance as yf

In [13]:

# Define parameters
N = 55 # Number of days for moving average and average true range
K1 = 2 # Multiple of ATR to use for stop loss
K2 = 0.5 # Multiple of ATR to use for adding to position
unit = 1000 # Unit size for position (in dollars)

# Load historical data
ticker = "RIG"
data = yf.download(ticker, period="1y")


[*********************100%***********************]  1 of 1 completed


In [14]:
from IPython.display import display
display(data)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-25,3.37,3.38,3.21,3.33,3.33,24075200
2022-02-28,3.28,3.55,3.28,3.54,3.54,18913400
2022-03-01,3.59,3.72,3.49,3.62,3.62,25462600
2022-03-02,3.52,3.77,3.52,3.76,3.76,25401500
2022-03-03,3.68,3.79,3.57,3.77,3.77,20410200
...,...,...,...,...,...,...
2023-02-17,7.33,7.41,7.00,7.01,7.01,23831400
2023-02-21,6.93,7.16,6.73,6.84,6.84,23685900
2023-02-22,6.41,6.50,5.61,6.10,6.10,53598800


In [15]:
# Calculate the True Range for each day
true_range = pd.DataFrame({
    "TR1": abs(data["High"] - data["Low"]),
    "TR2": abs(data["High"] - data["Adj Close"].shift(1)),
    "TR3": abs(data["Low"] - data["Adj Close"].shift(1))
})
data["TR"] = true_range.max(axis=1)

# Calculate the Average True Range (ATR)
data["ATR"] = data["TR"].rolling(window=N).mean()

In [16]:
display(data)

,Open,High,Low,Close,Adj Close,Volume,TR,ATR
Date,,,,,,,,
2022-02-25,3.37,3.38,3.21,3.33,3.33,24075200,0.17,NaN
2022-02-28,3.28,3.55,3.28,3.54,3.54,18913400,0.27,NaN
2022-03-01,3.59,3.72,3.49,3.62,3.62,25462600,0.23,NaN
2022-03-02,3.52,3.77,3.52,3.76,3.76,25401500,0.25,NaN
2022-03-03,3.68,3.79,3.57,3.77,3.77,20410200,0.22,NaN
...,...,...,...,...,...,...,...,...
2023-02-17,7.33,7.41,7.00,7.01,7.01,23831400,0.49,0.335636
2023-02-21,6.93,7.16,6.73,6.84,6.84,23685900,0.43,0.340000
2023-02-22,6.41,6.50,5.61,6.10,6.10,53598800,1.23,0.356364


In [17]:
# Calculate the entry and exit points for long positions
data["long_entry"] = data["High"].rolling(window=N).max().shift(1)
data["long_exit"] = data["Low"].rolling(window=N).min().shift(1)

# Calculate the entry and exit points for short positions
data["short_entry"] = data["Low"].rolling(window=N).min().shift(1)
data["short_exit"] = data["High"].rolling(window=N).max().shift(1)

In [18]:
display(data)

,Open,High,Low,Close,Adj Close,Volume,TR,ATR,long_entry,long_exit,short_entry,short_exit
Date,,,,,,,,,,,,
2022-02-25,3.37,3.38,3.21,3.33,3.33,24075200,0.17,NaN,NaN,NaN,NaN,NaN
2022-02-28,3.28,3.55,3.28,3.54,3.54,18913400,0.27,NaN,NaN,NaN,NaN,NaN
2022-03-01,3.59,3.72,3.49,3.62,3.62,25462600,0.23,NaN,NaN,NaN,NaN,NaN
2022-03-02,3.52,3.77,3.52,3.76,3.76,25401500,0.25,NaN,NaN,NaN,NaN,NaN
2022-03-03,3.68,3.79,3.57,3.77,3.77,20410200,0.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-17,7.33,7.41,7.00,7.01,7.01,23831400,0.49,0.335636,7.69,3.65,3.65,7.69
2023-02-21,6.93,7.16,6.73,6.84,6.84,23685900,0.43,0.340000,7.69,3.65,3.65,7.69
2023-02-22,6.41,6.50,5.61,6.10,6.10,53598800,1.23,0.356364,7.69,3.65,3.65,7.69


In [19]:
# Create a DataFrame to store the trading signals and the cumulative returns
signals = pd.DataFrame(index=data.index)
signals["signal"] = 0.0
signals["signal"] = np.where(data["Adj Close"] > data["long_entry"], 1.0, signals["signal"])
signals["signal"] = np.where(data["Adj Close"] < data["short_entry"], -1.0, signals["signal"])
signals["positions"] = signals["signal"].diff()
signals["returns"] = np.log(data["Adj Close"]/data["Adj Close"].shift(1))

In [20]:
display(signals)

,signal,positions,returns
Date,,,
2022-02-25,0.0,NaN,NaN
2022-02-28,0.0,0.0,0.061154
2022-03-01,0.0,0.0,0.022347
2022-03-02,0.0,0.0,0.037945
2022-03-03,0.0,0.0,0.002656
...,...,...,...
2023-02-17,0.0,0.0,-0.066231
2023-02-21,0.0,0.0,-0.024550
2023-02-22,0.0,0.0,-0.114499


In [21]:
# Backtest the trading signals
initial_capital = float(100000.0)
positions = pd.DataFrame(index=signals.index).fillna(0.0)
positions["RIG"] = 100 * signals["signal"]
portfolio = positions.multiply(data["Adj Close"], axis=0)
pos_diff = positions.diff()
portfolio["holdings"] = (positions.multiply(data["Adj Close"], axis=0)).sum(axis=1)
portfolio["cash"] = initial_capital - (pos_diff.multiply(data["Adj Close"], axis=0)).sum(axis=1).cumsum()
portfolio["total"] = portfolio["cash"] + portfolio["holdings"]
portfolio["returns"] = portfolio["total"].pct_change()

In [22]:
# Print the final portfolio value and the Sharpe ratio
print("Final Portfolio Value: ${:.2f}".format(portfolio["total"].iloc[-1]))

# indicates how much return an investor is receiving for each unit of risk taken
print("Sharpe Ratio: {:.2f}".format((portfolio["returns"].mean() / portfolio["returns"].std()) * np.sqrt(252)))

Final Portfolio Value: $99963.00
Sharpe Ratio: -0.48


In [ ]:
data['ATR'].ewm